Задача:
1. На основе файла electronic_devices.csv сгенерировать данные на 1 миллион строк c помощью sdv.

2. С помощью Spark прочитать данные, выбрать данные за «2024-09-03» и отфильтровать записи с одной дополнительной покупкой (столбец «addons»).
3. На основе полученных данных вычислить разницу между минимальной и максимальной ценой товара (столбец «unit_price»), разницу между минимальной и максимальной ценой заказа (столбец «total_price»). Данные необходимо группировать на основе пола, возраста, возраста и пола.
4. Данные можно собрать с помощью функции df.collect(), сохранять не нужно.
5. Необходимо получить среднее время выполнения агрегаций без кэширования после фильтра, а также с различными способами кэширования/сохранением контрольной точки.

In [35]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType, StructType, IntegerType, DateType, StructField
import pyspark.sql.functions as F
import pandas as pd
from sdv.metadata import Metadata
from sdv.single_table import GaussianCopulaSynthesizer

In [3]:
spark = SparkSession \
            .builder \
            .appName("spark_demo_2") \
            .master("local") \
            .getOrCreate()

24/12/17 16:30:17 WARN Utils: Your hostname, MacBook-Air-15.local resolves to a loopback address: 127.0.0.1; using 10.8.61.153 instead (on interface en0)
24/12/17 16:30:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/17 16:30:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark = SparkSession.builder \
    .master("local") \
    .config("spark.submit.deployMode", "client") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

24/12/17 16:30:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [16]:
spark.stop()

In [8]:
sdf = spark.read.csv('synthetic_electronic_devices.csv',
                    header='true')

In [9]:
sdf = sdf.withColumn('addons_splitted', f.split(f.col('addons'), ','))

In [10]:
sdf

DataFrame[customer_id: string, age: string, gender: string, loyalty_member: string, product_type: string, sku: string, rating: string, order_status: string, payment_method: string, total_price: string, unit_price: string, quantity: string, purchase_date: string, shipping_type: string, addons: string, addons_cnt: string, addons_splitted: array<string>]

In [163]:
agg_sdf = sdf.where((sdf['purchase_date']=='2024-09-03') & (f.size(sdf['addons_splitted']) == 1) & (sdf.addons != 'Null'))
agg_sdf.orderBy(f.col('age')).show()

+-----------+---+------+--------------+------------+-------+------+------------+--------------+-----------+----------+--------+-------------+-------------+-----------------+----------+-------------------+
|customer_id|age|gender|loyalty_member|product_type|    sku|rating|order_status|payment_method|total_price|unit_price|quantity|purchase_date|shipping_type|           addons|addons_cnt|    addons_splitted|
+-----------+---+------+--------------+------------+-------+------+------------+--------------+-----------+----------+--------+-------------+-------------+-----------------+----------+-------------------+
|       9626| 19|Female|            No|      Tablet|SKU1002|     3|   Completed|    Debit Card|   10610.57|    972.06|       2|   2024-09-03|    Expedited|        Accessory|     11.44|        [Accessory]|
|      15982| 19|  Male|           Yes|      Tablet| HDP456|     4|   Completed|          Cash|     221.86|    694.62|       5|   2024-09-03|      Express|        Accessory|      0

In [165]:
agg_sdf = agg_sdf.withColumn('unit_price_int', f.col('unit_price').cast(IntegerType()))

In [171]:
agg_sdf = agg_sdf.groupBy(f.col('age'), f.col('gender'), f.col('unit_price')) \
    .agg((f.max('unit_price_int') - f.min('unit_price_int'))
# agg_sdf.agg(f.min('unit_price_int')).show()

SyntaxError: incomplete input (3172972983.py, line 3)

In [169]:
agg_sdf.show()

+-----------+---+------+--------------+------------+-------+------+------------+--------------+-----------+----------+--------+-------------+-------------+-----------------+----------+-------------------+--------------+
|customer_id|age|gender|loyalty_member|product_type|    sku|rating|order_status|payment_method|total_price|unit_price|quantity|purchase_date|shipping_type|           addons|addons_cnt|    addons_splitted|unit_price_int|
+-----------+---+------+--------------+------------+-------+------+------------+--------------+-----------+----------+--------+-------------+-------------+-----------------+----------+-------------------+--------------+
|      17296| 64|Female|            No|      Tablet| HDP456|     4|   Completed|        Paypal|      644.0|    774.88|       2|   2024-09-03|    Expedited|        Accessory|      3.62|        [Accessory]|           774|
|      19687| 60|Female|            No|      Laptop| HDP456|     3|   Completed|   Credit Card|    5659.38|   1101.21|  

In [37]:
file_path = './electronic_devices.csv' 
df = pd.read_csv(file_path)
metadata = Metadata.detect_from_dataframes(
    data={
        'electronics': df
    })
metadata
metadata.validate()
metadata.save_to_json('my_final_metadata.json')

model = GaussianCopulaSynthesizer(metadata)
model.fit(data)

synthetic_data = model.sample(1000000)

synthetic_data.to_csv('synthetic_electronic_devices.csv', index=False)
print('1000000 of syntetics were created')

/opt/anaconda3/envs/sdv_env/lib/python3.10/site-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


NameError: name 'data' is not defined

In [39]:
file_path = '/Users/tuzhbagrigoriy/reps/some_hw/python_dz/HW11/gay.csv' 
df = pd.read_csv(file_path, encoding='utf8')
metadata = Metadata.detect_from_dataframes(
    data={
        'russian': df
    })
metadata
metadata.validate()
# metadata.save_to_json('my_final_metadata.json')

model = GaussianCopulaSynthesizer(metadata)
model.fit(data)

synthetic_data = model.sample(100)

synthetic_data.to_csv('synthetic_electronic_devices.csv', index=False)
print('1000000 of syntetics were created')

/opt/anaconda3/envs/sdv_env/lib/python3.10/site-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


NameError: name 'data' is not defined